In [4]:
%load_ext autoreload
%autoreload 2

import re, time, json,os
from app.utils import Helper
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = Helper.read_file("nse.txt")
config = Helper.load_json5(r"config\params.json5")

print(type(df))

<class 'str'>


In [ ]:
NSE_SYMBOL_REGEX = "\\|\\|([0-9A-Z$\\.]+.NS)"
NSE_PORTS = config["NSE_PORTS"]
SYMBOL_SET = set()

TODAY = datetime.now().strftime("%Y%m%d")


SAVE_PATH = f"{TODAY}"
Helper.create_dir(SAVE_PATH)

def write_file(filepath: str, content: str):
    with open(filepath, 'a', encoding="utf-8") as f:
        f.write(content + '\n')

def get_nse_symbol(ticker):
    symbol_list = re.findall(NSE_SYMBOL_REGEX,ticker, re.IGNORECASE)
    if symbol_list:
        symbol = symbol_list[0]
        SYMBOL_SET.add(symbol)
        return symbol
    return None


def get_nse_port(ticker):
    data_set = []
    print(ticker)
    
    for port_name, port_val in NSE_PORTS.items():
        port_regex = f"\\b{port_val}\\=(\\d+\\.?\\d*)~"
        if port_name == "NSE_DATETIME":
            port_regex = fr"{port_val}=(\d{{4}}-\d{{2}}-\d{{2}})\s(\d{{2}}:\d{{2}}:\d{{2}})~"
            
        if content:=re.findall(port_regex,ticker,re.IGNORECASE):
            print(f"{port_name}({port_val}): {content}")
            captured_data = content[0]
            captured_data = ",".join(captured_data) if isinstance(captured_data,tuple) else captured_data
        else:
            captured_data = "N/A,N/A" if port_name == "NSE_DATETIME" else "N/A"
        data_set.append(captured_data)
            
    return data_set
        
for ticker in df.split("\n"):
    
    symbol = get_nse_symbol(ticker)
    print("==================")
    if not symbol:
        continue
    print(symbol)
    data = get_nse_port(ticker)
    data_csv = ",".join(data)
    print(data_csv)
    
    symbol_data_path = os.path.join(SAVE_PATH,f"{symbol}.csv")
     
    if not os.path.exists(symbol_data_path):
        write_file(symbol_data_path, "Date,Time,LTP,BuyPrice,BuyQty,SellPrice,SellQty,LTQ")

    write_file(symbol_data_path, data_csv)
        
    
    


In [ ]:
 
# Ticker	Date	Time	LTP	BuyPrice	BuyQty	SellPrice	SellQty	LTQ	OpenInterest
# RIIL.NSE	19-05-2025	09:15:03	928.6	0	0	0	0	0	0
# RIIL.NSE	19-05-2025	09:15:03	929.7	926.2	14	929.7	7	1001	0
# RIIL.NSE	19-05-2025	09:15:03	927.95	926.2	14	929.7	7	0	0
# RIIL.NSE	19-05-2025	09:15:15	933.65	930.3	10	933.65	75	2079	0
# RIIL.NSE	19-05-2025	09:15:15	933.65	930.3	10	933.65	75	0	0
# RIIL.NSE	19-05-2025	09:15:25	934	931.45	10	934.45	5	322	0
# RIIL.NSE	19-05-2025	09:15:25	934	931.45	10	934.45	5	0	0
# RIIL.NSE	19-05-2025	09:15:34	931.35	932.2	10	933.95	1	1	0
# RIIL.NSE	19-05-2025	09:15:42	932.4	931.75	10	933.4	3	33	0
# RIIL.NSE	19-05-2025	09:15:51	931.95	931.65	14	933.5	5	275	0